<a href="https://colab.research.google.com/github/Xvixx/route-opt-project/blob/main/Route.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ติดตั้ง Google OR-Tools (สำหรับคำนวณ Optimization)
# และ Folium (สำหรับสร้างแผนที่สวยๆ)
!pip install ortools folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 24.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.31.1 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.31.1 w

In [8]:
import math
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import folium

# 1. กำหนดพิกัด (Lat, Long)
# จุดแรกคือ Depot (จุฬาฯ) ที่เหลือคือลูกค้า
locations = [
    (13.73826, 100.53273), # Chula (Depot)
    (13.74682, 100.53507), # Siam Paragon
    (13.72996, 100.53644), # Silom Complex
    (13.72120, 100.58359), # Thong Lo
    (13.67069, 100.60375), # Bang Na
    (13.81402, 100.56061), # Chatuchak
    (13.76495, 100.53830), # Victory Monument
]

# จำนวนรถที่เรามี
num_vehicles = 2
# จุดเริ่มต้น (Index 0 คือ Chula)
depot_index = 0

print(f"Data Prepared: {len(locations)-1} Customers, {num_vehicles} Vehicles")

Data Prepared: 6 Customers, 2 Vehicles


In [9]:
def haversine_distance(coord1, coord2):
    R = 6371  # รัศมีโลก (km)
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    # *** จุดที่แก้: ใส่ int() ครอบเพื่อปัดเศษทศนิยมทิ้ง ***
    return int(R * c * 1000)

def create_data_model():
    data = {}
    size = len(locations)
    dist_matrix = [[0] * size for _ in range(size)]
    for i in range(size):
        for j in range(size):
            dist_matrix[i][j] = haversine_distance(locations[i], locations[j])

    data['distance_matrix'] = dist_matrix
    data['num_vehicles'] = num_vehicles
    data['depot'] = depot_index
    return data

data = create_data_model()
print("Distance Matrix Created (Integer Version)!")

Distance Matrix Created (Integer Version)!


In [10]:
# สร้างตัวจัดการ Routing
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

# บอก Model ว่าค่าใช้จ่ายในการเดินทางคือ "ระยะทาง"
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# ---------------------------------------------------------
# ส่วนที่เพิ่มเข้ามา: บังคับให้กระจายงาน (Load Balancing)
# ---------------------------------------------------------
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,       # no slack
    300000,  # ระยะทางสูงสุดที่รถ 1 คันวิ่งได้ (หน่วยเมตร) ตั้งเผื่อไว้เยอะๆ
    True,    # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)

# บรรทัดนี้คือพระเอก! สั่งให้พยายามทำให้รถที่วิ่งเยอะที่สุด วิ่งให้น้อยลง
# (มันจะบีบให้ต้องเอารถอีกคันมาช่วยวิ่ง)
distance_dimension.SetGlobalSpanCostCoefficient(100)
# ---------------------------------------------------------

# ตั้งค่า Solver
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# สั่งคำนวณ!
solution = routing.SolveWithParameters(search_parameters)

if solution:
    print("Solution Found! (Load Balanced)")
else:
    print("No solution found.")

Solution Found! (Load Balanced)


In [11]:
# สร้างแผนที่ปักหมุดที่กรุงเทพฯ
m = folium.Map(location=[13.7563, 100.5018], zoom_start=12)

# สีของเส้นทางรถแต่ละคัน
colors = ['blue', 'red', 'green', 'purple']

if solution:
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route_coords = []
        route_text = f"Vehicle {vehicle_id} Route: "

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_coords.append(locations[node_index])
            route_text += f"Node {node_index} -> "

            # ปักหมุด (Marker)
            folium.Marker(
                locations[node_index],
                popup=f"Point {node_index}",
                icon=folium.Icon(color=colors[vehicle_id % len(colors)])
            ).add_to(m)

            index = solution.Value(routing.NextVar(index))

        # เพิ่มจุดสุดท้าย (กลับ Depot)
        node_index = manager.IndexToNode(index)
        route_coords.append(locations[node_index])
        route_text += f"End"
        print(route_text)

        # วาดเส้นทาง (Polyline)
        folium.PolyLine(
            route_coords,
            color=colors[vehicle_id % len(colors)],
            weight=5,
            opacity=0.8
        ).add_to(m)

# แสดงแผนที่
m

Vehicle 0 Route: Node 0 -> Node 2 -> Node 5 -> Node 6 -> Node 1 -> End
Vehicle 1 Route: Node 0 -> Node 3 -> Node 4 -> End
